In [159]:
# 引入所需的套件
import jieba.posseg as pseg
import pandas as pd
import string
import keras
import numpy as np
import pickle
from sklearn.model_selection \
    import train_test_split
from keras import layers

In [9]:
# 結巴切詞函數
def jieba_tokenizer(text):
    words = pseg.cut(text)
    return ' '.join([
        word for word, flag in words if flag != 'x'])

In [10]:
# 加入結巴切詞轉為list的函數
def jieba_to_list(text):
    seg_text = jieba_tokenizer(text)
    seg_list = seg_text.split(' ')
    return seg_list

In [11]:
# 檢查資料
data = pd.read_excel('/Users/wangyuda/Desktop/文字探勘/Model/all.xlsx')

In [12]:
# 去除N/A的公司資料
# N/A的原因在於這些公司在2019年間下市
data = data.dropna(inplace = False)

In [8]:
# 去除公司簡稱後面的空白
# 去除事件內容數字間的 ','，讓數字資訊更精確
for i in range(len(data)):
    data.iloc[i,1] = data.iloc[i,1].rstrip()
    data.iloc[i,7] = data.iloc[i,7].replace(',', '')

In [14]:
# 將「TCRI」 與「年」、「月」分開
TCRI = []
year = []
month = []
for i in range(len(data)):
    if type(data.iloc[i,3]) == str:
        TCRI.append(data.iloc[i,3][0])
        year.append(data.iloc[i,3][2:6])
        month.append(data.iloc[i,3][7:9])
    else:
        TCRI.append('N/A')
        year.append('N/A')
        month.append('N/A')

data.insert(3, 'TCRI-發生月', month)
data.insert(3, 'TCRI-發生年', year)
data.insert(3, 'TCRI', TCRI)


In [163]:
data

,個股代號,公司簡稱,事件日,TCRI,TCRI-發生年,TCRI-發生月,TCRI(年/月),事件強度,大事件類別,小事件類別,事件內容,切詞結果,文字向量
0,1218,泰山,20190101,6,2018,09,6(2018/09),0,M_經營層,MT06高管異動,發言人林俐婉內部調動，由江巍峰接任。。,"[發言, 人, 林俐婉, 內部, 調動, 由, 江巍峰, 接任]","[453, 31, 36927, 80, 133, 26, 19247, 88]"
1,1503,士電,20190101,4,2018,09,4(2018/09),0,M_經營層,MT06高管異動,內部稽核主管林志強內部調動，由莊文清接任。。,"[內部, 稽核, 主管, 林志強, 內部, 調動, 由, 莊文清, 接任]","[80, 368, 132, 36928, 80, 133, 26, 36929, 88]"
2,1504,東元,20190101,4,2018,09,4(2018/09),0,M_經營層,MT06高管異動,會計主管藍俊雄內部調動，由林鴻名接任。。,"[會計, 主管, 藍俊雄, 內部, 調動, 由, 林鴻名, 接任]","[419, 132, 36930, 80, 133, 26, 36931, 88]"
3,1709,和益,20190101,5,2018,09,5(2018/09),0,M_經營層,MT06高管異動,內部稽核主管游本詮內部調動，由曾筱茜接任。。,"[內部, 稽核, 主管, 游本, 詮內部, 調動, 由, 曾筱茜, 接任]","[80, 368, 132, 13753, 24477, 133, 26, 24478, 88]"
4,1721,三晃,20190101,7,2018,09,7(2018/09),0,M_經營層,MT06高管異動,財務經理洪廷宜內部調動，由王婷渝接任。。,"[財務, 經理, 洪廷宜, 內部, 調動, 由, 王婷渝, 接任]","[126, 151, 36932, 80, 133, 26, 36933, 88]"
5,1817,凱撒衛,20190101,6,2018,09,6(2018/09),0,M_經營層,MT06高管異動,研發主管吳政峰內部調動，由朱清立接任。。,"[研發, 主管, 吳政峰, 內部, 調動, 由, 朱清立, 接任]","[276, 132, 36934, 80, 133, 26, 36935, 88]"
6,2064,晉椿,20190101,7,2018,09,7(2018/09),-1,M_經營層,MT06高管異動,總經理高進義離職，由陳譽接任。。發言人高進義離職，由陳譽接任。。,"[總經, 理高, 進義, 離職, 由, 陳譽, 接任, 發言人高進義, 離職, 由, 陳譽,...","[2472, 19248, 36936, 236, 26, 13754, 88, 24479..."
7,2207,和泰車,20190101,4,2018,09,4(2018/09),0,M_經營層,MT02董監異動,改派1董。董事大野勝仁(豐田自動車代表)卸任。董事長沼一生(豐田自動車代表)新任。,"[改派, 1, 董, 董事, 大野, 勝仁, 豐田, 自動車, 代表, 卸任, 董事, 長,...","[450, 38, 83, 4, 24480, 19249, 5280, 12125, 32..."
8,2330,台積電,20190101,1,2018,09,1(2018/09),-1,M_經營層,MT02董監異動,辭任1董。獨立董事湯馬斯?延吉布斯卸任。,"[辭任, 1, 董, 獨立, 董事, 湯馬斯, 延, 吉布斯, 卸任]","[573, 38, 83, 56, 4, 36937, 2616, 36938, 63]"
9,2357,華碩,20190101,2,2018,09,2(2018/09),0,M_經營層,MT06高管異動,總經理沈振來內部調動，由胡書賓接任。。,"[總, 經理, 沈振來, 內部, 調動, 由, 胡書賓, 接任]","[52, 151, 24482, 80, 133, 26, 36939, 88]"


In [134]:
# 用Apply切詞
seg = []
seg = data.loc[:, '事件內容'].apply(jieba_to_list)

In [ ]:
# # 用迴圈切詞
# segment = []
# for i in range(len(data)):
#     seg_text = jieba_tokenizer(data.iloc[i,10])
#     seg_list = seg_text.split(' ')
#     segment.append(seg_list)

In [136]:
# 將切詞結果併入dataframe
data.insert(11, '切詞結果', seg)

In [25]:
# 建立文字向量用的字典
MAX_NUM_WORDS = 100000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words = MAX_NUM_WORDS)

tokenizer.fit_on_texts(seg)
seg_to_number = tokenizer.texts_to_sequences(seg)

In [89]:
# 檢查數字轉換結果
print(len(seg_to_number))
seg_to_number[:4]

22775


list

In [90]:
# 將文字向量併入data
data.insert(12, '文字向量', seg_to_number)

In [27]:
# 檢查數字轉換結果
for seq in seg_to_number[:1]:
    print([tokenizer.index_word[idx] for idx in seq])

['發言', '人', '林俐婉', '內部', '調動', '由', '江巍峰', '接任']


In [91]:
# 確認各新聞文字字數狀況
for seq in data.iloc[:10,12]:
    print(len(seq), seq[:5], ' ...')

8 [453, 31, 36927, 80, 133]  ...
9 [80, 368, 132, 36928, 80]  ...
8 [419, 132, 36930, 80, 133]  ...
9 [80, 368, 132, 13753, 24477]  ...
8 [126, 151, 36932, 80, 133]  ...
8 [276, 132, 36934, 80, 133]  ...
12 [2472, 19248, 36936, 236, 26]  ...
18 [450, 38, 83, 4, 24480]  ...
9 [573, 38, 83, 56, 4]  ...
8 [52, 151, 24482, 80, 133]  ...


In [97]:
# 確認最大詞數
mean_seq_len = sum([len(seq) for seq in data.iloc[:,12]]) / len(data)
mean_seq_len

63.22656421514819

In [98]:
# 將各新聞文字字數固定
MAX_SEQUENCE_LENGTH = 63
seg_short = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(seg_to_number, 
                   maxlen=MAX_SEQUENCE_LENGTH)

# 處理 Y (大事件、小事件）

In [117]:
# 大事件轉換成數字
# 定義每一個分類對應到的索引數字
label_to_index = {
    'A_會計/財報分析': 0, 
    'F_市場交易': 1, 
    'I_產業前景': 2,
    'M_經營層': 3,
    'R_危機': 4
}

# 將分類標籤對應到剛定義的數字
y_big = data.iloc[:, 8].apply(lambda x: label_to_index[x])
y_big = np.asarray(y_big).astype('float32')

# 測試結果
y_big[:5]

# One Hot Encoding
y_big = keras.utils.to_categorical(y_big)

# 測試結果
y_big[:5]


array([3., 3., 3., 3., 3.], dtype=float32)

In [119]:
# 小事件轉換成數字
# 建立文字向量用的字典
MAX_NUM_WORDS_1 = 1000
tokenizer_1 = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words = MAX_NUM_WORDS_1)

small = []
for i in range(len(data)):
    data.iloc[i, 9] = data.iloc[i, 9].replace('_', '')
    small.append(data.iloc[i, 9])

tokenizer_1.fit_on_texts(small)
y_small = tokenizer_1.texts_to_sequences(small)

for i in range(len(data)):
    y_small[i] = y_small[i][0]
    
y_small = np.asarray(y_small).astype('float32')

# 測試結果
y_small[:5]

# One Hot Encoding
y_small = keras.utils.to_categorical(y_small)

# 測試結果
y_small[:5]

array([7., 7., 7., 7., 7.], dtype=float32)

In [ ]:
# # 小事件轉換成數字
# # 用土法煉鋼法
# label_to_index_1 = {
#     '1A_內部詐欺－未經授權行為': 0,
#     '1B_內部詐欺－竊盜與詐欺': 1,
#     '3B_工作場所－環境安全': 2,
#     '4A_客戶及產品－揭露義務': 3,
#     '4B_客戶及產品－不當行為': 4
#     '4C_客戶及產品－產品瑕疵':
#     '4D_客戶及產品－暴險':
#     '7A_作業流程－交易紀錄':
#     '7B_作業流程－監控與報告':
#     '7C_作業流程－客戶文件':
#     '7D_作業流程－帳戶管理':
#     'AF01_財報未出具或遲交':
#     'AF02_變更會計師事務所':
#     'AF03_會計估計變動/會計政策變更':
#     'AF04_市場傳聞造假(財報舞弊)':
#     'AF05_財務警示':
#     'AF06_資產減損':
#     'AF07_財報重編':
#     'AF09_非無保留意見':
#     'AI01_延遲公告':
#     'AI02_依主管機關要求補充公告':
#     'AI03_公告不實':
#     'AI04_自結數有重大差異':
#     'FS01_股價操縱/內部人交易/庫藏股':
#     'FS02_股價暴跌或異常':
#     '':
#     '':
#     '':
#     '':
#     '':
#     '':
#     '':
#     '':
# }

# # 將分類標籤對應到剛定義的數字
# y_small = data.iloc[:, 9].apply(lambda x: label_to_index_1[x])

# y_small = np.asarray(y_small).astype('float32')

# # 測試結果
# y_small[:5]

In [146]:
# 強度轉成array
y_strength = data.iloc[:, 7] + 3
y_strength = np.asarray(y_strength).astype('float32')
y_strength = keras.utils.to_categorical(y_strength)
print(y_strength[:5])


[[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


In [ ]:
# 將切割完成的list獨立存到一個可繼續讀取的檔案
file = open(r"/Users/wangyuda/Desktop/文字探勘/Model/list.bin","wb")
pickle.dump(data.iloc[:,11],file)
file.close()

In [83]:
# 讀取list檔
list_1 = []
file = open(r"/Users/wangyuda/Desktop/文字探勘/Model/list.bin","rb")
list_1 = pickle.load(file)

In [149]:
# 切割資料為兩部分：訓練部分與驗證部分
VALIDATION_RATIO = 0.1
# 小彩蛋
RANDOM_STATE = 9527

x = seg_short

x_train, x_val, \
yb_train, yb_val, \
ys_train, ys_val = \
    train_test_split(
        x, y_big, y_small, 
        test_size=VALIDATION_RATIO, 
        random_state=RANDOM_STATE
)

In [169]:
y_small[:5]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0.

In [170]:
ys_train.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'